In [1]:
using JuMP, HiGHS, LinearAlgebra, CSV, DataFrames

In [2]:
function inner_problem_primal(x, Γ, mean_ret, std_ret)
    m = JuMP.Model(HiGHS.Optimizer)
    set_silent(m)
    N = length(x)
    @variables(m,
    begin
    z[1:N] ≥ 0
    r[1:N] ≥ 0
    end)
    @constraint(m, con1, sum(z) == Γ)
    @constraint(m, con2[n=1:N], z[n] ≤ 1)
    @constraint(m, con3[n=1:N], r[n] + std_ret[n]*z[n] ≥  mean_ret[n])
    @constraint(m, con4[n=1:N], -r[n] + std_ret[n]*z[n] ≥ -mean_ret[n])
    @objective(m, Min, r ⋅ x)
    optimize!(m)
    return value.(z), value.(r), dual.(con1), dual.(con2), dual.(con3), dual.(con4), objective_value(m)
end

inner_problem_primal (generic function with 1 method)

In [3]:
function inner_problem_dual(x, Γ, mean_ret, std_ret)
    m = JuMP.Model(HiGHS.Optimizer)
    set_silent(m)
    N = length(x)
    @variables(m,
    begin
    Δ_p[1:N] ≥ 0
    Δ_n[1:N] ≥ 0
    λ[1:N] ≤ 0
    β
    end)
    @constraint(m, con1[n=1:N], -Δ_p[n] + Δ_n[n] ≥ -x[n])
    @constraint(m, con2[n=1:N], -σ[n]*(Δ_p[n] + Δ_n[n]) - β - λ[n] ≥ 0)
    @objective(m, Max, Γ*β + sum(λ) + sum(mean_ret .* (Δ_p .- Δ_n)))
    optimize!(m)
    return value.(Δ_p), value.(Δ_n), value.(λ), value.(β), dual.(con1), dual.(con2), objective_value(m)
end

inner_problem_dual (generic function with 1 method)

In [4]:
returns_df = CSV.read("../projeto final/returns.csv", DataFrame)

,B3SA3.SA,PETR4.SA,VALE3.SA,ITUB4.SA,BBAS3.SA,USIM5.SA
,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.0141026,0.0583039,-0.0705882,-0.00362438,-0.0145186,-0.0710723
2,-0.0169051,-0.00300501,0.0300819,0.00727513,-0.0204187,0.00805369
3,-0.00529101,-0.00167448,-0.0294926,-0.0019698,-0.023219,-0.0372836
4,-0.0571809,0.0925864,0.00476687,-0.0282895,0.0486224,0.0165975
5,-0.0211566,0.0119742,0.0512973,-0.00778605,-0.00592478,0.0435374
6,-0.0504323,-0.00758495,0.0369482,-0.0180826,-0.0124385,-0.0286832
7,0.0508346,0.0250688,-0.0217598,0.0218902,0.0367358,0.0241611
8,0.0137184,0.0214733,0.0,0.00816049,0.0174639,-0.00524246
9,0.0420228,0.101314,0.0305853,0.0475548,0.111443,0.0289855


In [5]:
using Statistics

mat = Matrix{Float64}(returns_df)
r̄ = mean(mat, dims=1)
σ = std(mat, dims=1)

@show r̄, σ

(r̄, σ) = ([0.0017683520170361459 0.0004364872922691337 0.00013768814649423763 0.0006182894036464786 0.00035927071713961735 0.0008174884442208024], [0.07015487425825495 0.03303773882289478 0.027870781429468736 0.026784813296254643 0.026318255172717457 0.03411506037258844])


([0.0017683520170361459 0.0004364872922691337 … 0.00035927071713961735 0.0008174884442208024], [0.07015487425825495 0.03303773882289478 … 0.026318255172717457 0.03411506037258844])

In [59]:
r̄ = 1 .+ r̄

1×6 Matrix{Float64}:
 1.00177  1.00044  1.00014  1.00062  1.00036  1.00082

In [6]:
z, r, D_β, D_λ, D_Δ_p, D_Δ_n, obj = inner_problem_primal([10/6 for i in 1:6], 1, r̄, σ)

([0.02520640277290595, 0.013211778645294362, 0.9008845445556758, 0.023083580863822367, 0.013651008198752156, 0.02396268496354932], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], -0.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-0.0, -0.0, 0.0, -0.0, -0.0, -0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 0.0)

In [7]:
Δ_p, Δ_n, λ, β, D_r, D_z, D_obj = inner_problem_dual([10/6 for i in 1:6], 1, r̄, σ)

([-0.0, -0.0, -0.0, -0.0, -0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], -0.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.05897774124111637, 0.125237869425226, 0.14845568759086225, 0.15447470083298953, 0.15721315845799652, 0.3556408424518094], 0.0)

In [8]:
println(D_Δ_p == Δ_p)
println(D_Δ_n == Δ_n)
println(D_λ == λ)
println(D_β == β)
println(D_r == r)
println(D_z == z)
println(D_obj == obj)

true
true
true
true
true
false
true
